# Debug Links

Trying to debug why GoodReads links aren't working.

In [1]:
import pandas as pd
from bookdata.db import db_url

## Compare Book IDs

In [5]:
db_ids = pd.read_sql_table('book_ids', db_url(), schema='gr')
db_ids = db_ids.set_index('gr_book_id').rename(columns={'gr_isbn': 'gr_isbn10'})
db_ids.head()

,gr_book_rid,gr_work_id,gr_asin,gr_isbn10,gr_isbn13
gr_book_id,,,,,
5333265,1,5400751.0,None,0312853122,9780312853129
1333909,2,1323437.0,None,0743509986,9780743509985
7327624,3,8948723.0,B00071IKUY,None,None
6066819,4,6243154.0,None,0743294297,9780743294294
287140,5,278577.0,None,0850308712,9780850308716


In [3]:
pq_ids = pd.read_parquet('goodreads/gr-book-ids.parquet')
pq_ids = pq_ids.set_index('book_id')
pq_ids.head()

,work_id,isbn10,isbn13,asin
book_id,,,,
5333265,5400751.0,0312853122,9780312853129,None
1333909,1323437.0,0743509986,9780743509985,None
7327624,8948723.0,None,None,B00071IKUY
6066819,6243154.0,0743294297,9780743294294,None
287140,278577.0,0850308712,9780850308716,None


In [6]:
both_ids = db_ids.join(pq_ids)
both_ids.head()

,gr_book_rid,gr_work_id,gr_asin,gr_isbn10,gr_isbn13,work_id,isbn10,isbn13,asin
gr_book_id,,,,,,,,,
5333265,1,5400751.0,None,0312853122,9780312853129,5400751.0,0312853122,9780312853129,None
1333909,2,1323437.0,None,0743509986,9780743509985,1323437.0,0743509986,9780743509985,None
7327624,3,8948723.0,B00071IKUY,None,None,8948723.0,None,None,B00071IKUY
6066819,4,6243154.0,None,0743294297,9780743294294,6243154.0,0743294297,9780743294294,None
287140,5,278577.0,None,0850308712,9780850308716,278577.0,0850308712,9780850308716,None


Look for different ISBNs

In [13]:
def isbn_diff(field, df=both_ids):
    mask = df[f'gr_{field}'].isnull() & df[field].isnull()
    mask |= df[f'gr_{field}'] == df[field]
    return df[~mask]

In [14]:
isbn_diff('isbn10')

,gr_book_rid,gr_work_id,gr_asin,gr_isbn10,gr_isbn13,work_id,isbn10,isbn13,asin
gr_book_id,,,,,,,,,
24717704,1178,23589581.0,None,B00F8LQRDU,None,23589581.0,008,None,None
8013976,1704,12568197.0,None,951230922x,None,12568197.0,951230922X,None,None
28099039,3118,48103788.0,None,B0006PGPH6,None,48103788.0,00066,None,None
22743798,3378,42286486.0,None,B00LMDSA92,None,42286486.0,0092,None,None
23592581,3699,23909277.0,None,B00B7TG2DC,None,23909277.0,0072,None,None
...,...,...,...,...,...,...,...,...,...
8882157,2358548,485998.0,None,ISBN043590,None,485998.0,043590,None,None
25597821,2358916,42786177.0,None,B00Y2I8DB4,None,42786177.0,00284,None,None
15851621,2358951,21598945.0,None,Published:,None,21598945.0,,None,None


In [15]:
isbn_diff('isbn13')

,gr_book_rid,gr_work_id,gr_asin,gr_isbn10,gr_isbn13,work_id,isbn10,isbn13,asin
gr_book_id,,,,,,,,,
16081442,201,21880573.0,None,None,B009N841IM,21880573.0,None,009841,None
17859771,1185,25002827.0,None,None,B00CJOCEPO,25002827.0,None,00,None
18622540,1286,26408939.0,None,None,B00FIPTFYO,26408939.0,None,00,None
7672457,1428,10288208.0,None,9654824795,B007V62UAY,10288208.0,9654824795,00762,None
22994886,2113,45759196.0,None,None,B00IRNKY8K,45759196.0,None,008,None
...,...,...,...,...,...,...,...,...,...
20518296,2358575,36374558.0,None,9788415336,B00F58LO02,36374558.0,9788415336,005802,None
18631369,2358880,26424641.0,None,9780992029,B00FI1S9Y0,26424641.0,9780992029,00190,None
13091693,2359744,4781036.0,None,None,B002BU5QYW,4781036.0,None,0025,None


In [16]:
isbn_diff('asin')

,gr_book_rid,gr_work_id,gr_asin,gr_isbn10,gr_isbn13,work_id,isbn10,isbn13,asin
gr_book_id,,,,,,,,,
31346587,3069,50116650.0,RayJayPerr,None,None,50116650.0,None,None,RAYJAYPERR
13115749,3914,18289742.0,BNID:29400,None,None,18289742.0,None,None,BNID29400
25798264,9049,44673889.0,UnderACorn,None,None,44673889.0,None,None,UNDERACORN
11853906,26169,16810092.0,01988/5812,None,None,16810092.0,None,None,019885812
3140368,26722,3171815.0,LCCBT1375.,None,None,3171815.0,None,None,LCCBT1375
...,...,...,...,...,...,...,...,...,...
22838579,2317329,3652165.0,Simon&amp;Schu,None,None,3652165.0,None,None,SIMONAMPSCHU
29350161,2317396,49589581.0,ASIN:B0192,None,None,49589581.0,None,None,ASINB0192
32316418,2318956,45666384.0,/B01D27EX3,None,None,45666384.0,None,None,B01D27EX3


Conclusion: we don't have that many books with different ISBNs.

## Look at Book Clustering

Let's scan the book clustering table in the DB:

In [28]:
db_clusters = pd.read_sql_table('book_cluster', db_url(), schema='gr')
db_clusters = db_clusters.set_index('gr_book_id')
db_clusters.sort_index(inplace=True)
db_clusters

,cluster
gr_book_id,
1,167476
2,2044243
3,108089
4,108089
5,108089
...,...
36519806,6039792
36525806,5733872
36525824,5864550


And load from Parquet:

In [27]:
pq_links = pd.read_parquet('goodreads/gr-book-link.parquet').set_index('book_id')
pq_links.sort_index(inplace=True)
pq_links

,work_id,cluster
book_id,,
1,41335427.0,100594612
2,2809203.0,103432670
3,4640799.0,100620223
4,6231171.0,100620223
5,2402163.0,100620223
...,...,...
36525824,57882612.0,457882612
36526448,58250451.0,458250451
36529314,58253323.0,458253323


In [29]:
pq_links['ccode'] = pq_links['cluster'] // 100000000
pq_links[~pq_links['ccode'].isin([4, 5])]

,work_id,cluster,ccode
book_id,,,
1,41335427.0,100594612,1
2,2809203.0,103432670,1
3,4640799.0,100620223,1
4,6231171.0,100620223,1
5,2402163.0,100620223,1
...,...,...,...
36498328,827903.0,103602696,1
36498955,45629111.0,108702505,1
36499229,1882486.0,100667266,1


We have fewer books resolving to a non-GR cluster, but not *that* many.

## Load Genders

Let's load the gender files.

In [47]:
db_genders = pd.read_sql_table('cluster_first_author_gender', db_url())
db_genders = db_genders.set_index('cluster')['gender'].astype('category')
db_genders.cat.rename_categories({'no-loc-author': 'no-book-author', 'no-viaf-author': 'no-author-rec'}, inplace=True)
db_genders.cat.add_categories('no-book', inplace=True)
db_genders.head()

cluster
10935495           female
4464609            female
10801730           female
756002             female
9076729     no-author-rec
Name: gender, dtype: category
Categories (7, object): ['ambiguous', 'female', 'male', 'no-book-author', 'no-author-rec', 'unknown', 'no-book']

In [48]:
pq_genders = pd.read_parquet('book-links/cluster-genders.parquet')
pq_genders = pq_genders.set_index('cluster')['gender'].astype('category')
pq_genders.head()

cluster
100109686         ambiguous
100027659         ambiguous
100025369         ambiguous
401752021    no-book-author
900000005           no-book
Name: gender, dtype: category
Categories (7, object): ['ambiguous', 'female', 'male', 'no-author-rec', 'no-book', 'no-book-author', 'unknown']

Now get genders:

In [49]:
db_gr_genders = db_clusters.join(db_genders, on='cluster', how='left')
db_gr_genders

,cluster,gender
gr_book_id,,
1,167476,ambiguous
2,2044243,female
3,108089,ambiguous
4,108089,ambiguous
5,108089,ambiguous
...,...,...
36519806,6039792,no-book-author
36525806,5733872,no-book-author
36525824,5864550,no-book-author


In [50]:
pq_gr_genders = pq_links.join(pq_genders, on='cluster', how='left')
pq_gr_genders

,work_id,cluster,ccode,gender
book_id,,,,
1,41335427.0,100594612,1,ambiguous
2,2809203.0,103432670,1,female
3,4640799.0,100620223,1,ambiguous
4,6231171.0,100620223,1,ambiguous
5,2402163.0,100620223,1,ambiguous
...,...,...,...,...
36525824,57882612.0,457882612,4,no-book-author
36526448,58250451.0,458250451,4,no-book-author
36529314,58253323.0,458253323,4,no-book-author


In [51]:
genders = pq_gr_genders.join(db_gr_genders, lsuffix='_pq', rsuffix='_db')
genders

,work_id,cluster_pq,ccode,gender_pq,cluster_db,gender_db
book_id,,,,,,
1,41335427.0,100594612,1,ambiguous,167476.0,ambiguous
2,2809203.0,103432670,1,female,2044243.0,female
3,4640799.0,100620223,1,ambiguous,108089.0,ambiguous
4,6231171.0,100620223,1,ambiguous,108089.0,ambiguous
5,2402163.0,100620223,1,ambiguous,108089.0,ambiguous
...,...,...,...,...,...,...
36525824,57882612.0,457882612,4,no-book-author,5864550.0,no-book-author
36526448,58250451.0,458250451,4,no-book-author,5634754.0,no-book-author
36529314,58253323.0,458253323,4,no-book-author,NaN,NaN


In [52]:
genders[genders.eval('gender_db.notnull() & (gender_pq != gender_db)')]

,work_id,cluster_pq,ccode,gender_pq,cluster_db,gender_db
book_id,,,,,,
323,4641.0,109192851,1,ambiguous,3921144.0,female
374,4676.0,108011562,1,male,5572225.0,unknown
667,287946.0,100109686,1,ambiguous,2465344.0,female
685,1433491.0,101086930,1,ambiguous,1039465.0,male
912,3366043.0,100543430,1,ambiguous,4394172.0,male
...,...,...,...,...,...,...
36426210,27792495.0,100109686,1,ambiguous,6175929.0,no-book-author
36430330,27382963.0,100109686,1,ambiguous,1540752.0,female
36432318,41420870.0,100209221,1,ambiguous,6114528.0,no-book-author
